In [ ]:
using ProgressMeter, PyCall, PyPlot, Cairo, Images, HDF5, MultivariateStats, Interpolations, Lasso, Distributions, ImageFiltering
using _Data
using  NaNStatistics, Statistics
#using ScikitLearn
include("../project_place_cell/functions/func_map.jl")
include("../Decoding/Decoder_Functions.jl")
include("../Decoding/Decoder_Pipeline.jl")
np = pyimport("numpy")



rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["axes.spines.top"] = false;
rc_params["axes.spines.right"] = false;

rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;

rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
datasets_corner_cue = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 25, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 25, "jen"],
    ["20220416_160516", 6, "jen"]
];

chuyu_server = [4, 9, 5, 2, 9, 4, 6]

n_datasets = length(datasets_corner_cue)
lengths = [90, 90, 90, 89, 90, 90, 90, 90, 90];
activity_bins = 7
activity_shift = 4
n_pos = 60 #number of bins in long side
long_axis_in_mm = 47 #47 for rectangular, 33 for others
use_amount = 1000

# EDF1 e

## one dataset

In [ ]:

i=1

experiment_filename = datasets_corner_cue[i][1]
server = datasets_corner_cue[i][2]
experimenter = datasets_corner_cue[i][3]

ds_Lorenz = Dataset(experiment_filename, "lorenz", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
ds_Chuyu = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(chuyu_server[i])" ? "/data" : "/nfs/data$(chuyu_server[i])")

ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")


In [ ]:

C, heading, img_bg, y_fish, x_offset, x_fish, y_offset = h5open(ds, "behavior.h5"; raw = true) do file
    read(file, "C"),
    read(file, "heading"),
    read(file, "img_bg"),
    read(file, "fish_yolk_y"),
    read(file, "offset_x"),
    read(file, "fish_yolk_x"),
    read(file, "offset_y")
end;



# orientation-corrected fish location (time binned)
position_file = h5open(joinpath(data_path(ds_Chuyu), "for_place_calculation_chamber_geometry_$(experiment_filename)_n60.h5"))
    chamber_roi = read(position_file,"chamber_roi")
    x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
    y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
    speed_mm_s = read(position_file, "speed_mm_s")
    loc_digital = read(position_file, "loc_digital")
    x_digital = read(position_file, "x_digital")
    y_digital = read(position_file, "y_digital")
    x_bins = read(position_file, "x_bins")
    y_bins = read(position_file, "y_bins")
close(position_file)

moving_valid = speed_mm_s .> 0.1;

min_x = floor(Int64, minimum(x_fish_sweep_mean));
max_x = floor(Int64, maximum(x_fish_sweep_mean));

min_y = floor(Int64, minimum(y_fish_sweep_mean));
max_y = floor(Int64, maximum(y_fish_sweep_mean));

interval = maximum([(max_y-min_y+2)/n_pos,(max_x-min_x+2)/n_pos])


file = h5open(joinpath(data_path(ds_Chuyu), "neuron_spatial_info_0_90_chamber_geometry_$(experiment_filename)_sigma1_n60.h5"), "r") #_whole #spatial_info_4 done on merged cells
    valid_neurons = HDF5.readmmap(file["valid_neurons"])
    specificity_shuffle_z = HDF5.readmmap(file["specificity_shuffle_z"])
    specificity_population_z = HDF5.readmmap(file["specificity_population_z"])
    specificity = HDF5.readmmap(file["specificity"])
    bool_index = HDF5.readmmap(file["bool_index"])
close(file)

bool_index = BitArray(bool_index)



file = h5open(joinpath(data_path(ds_Chuyu), "NMF_merge.h5"), "r")
    A_dFF = HDF5.readmmap(file["A_dFF"])

    z_all = HDF5.readmmap(file["Z_all"])
    centroid_x_all = HDF5.readmmap(file["X_all"])
    centroid_y_all = HDF5.readmmap(file["Y_all"])
close(file)

n_neurons = size(A_dFF, 2)
n_sweeps = size(A_dFF, 1)



function bin_to_px(x, y, offset=true)
    if offset
        return (x .- 0.5) .* interval .+ (min_x-1), (y .- 0.5) .* interval .+ (min_y-1)
    else
        return (x .- 0.5) .* interval, (y .- 0.5) .* interval
    end
end

function px_to_bin(x, y)
    return 0.5 .+ ((x .- (min_x-1)) ./ interval), 0.5 .+ ((y .- (min_y-1)) ./ interval)
end

x_in_bins, y_in_bins = px_to_bin(x_fish_sweep_mean, y_fish_sweep_mean);

try
    mkdir(data_path(ds_Lorenz))    
catch
    println("save path exists")
end

place_candidates_unique = Decoder.get_top_neurons(use_amount, specificity_population_z, specificity_shuffle_z);

In [ ]:
function digital2map(which_loc::Vector,n_pos::Int)
    x = mod.(which_loc, n_pos)
    y = Int.((which_loc .- x) ./ n_pos) .+ 1
    return x,y
end

In [ ]:
# compute place maps
mean_map_all = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
count_map_all = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
@showprogress for neuron_idx in 1:size(A_dFF, 2)

    neural_activity, which_loc = MAP.valid_activity_loc(A_dFF[:, neuron_idx], bool_index, loc_digital)
    mean_map_all[:, :, neuron_idx], count_map_all[:, :, neuron_idx], summed = MAP.calculate_map_direct(neural_activity, which_loc, n_pos; at_least_visit = 0, use_gaussian_filter=true, sigma=1, filter_mask = nothing)

end

In [ ]:
# helper function
function whether_in(vector, collection)
    return [x in collection for x in vector]
end


# compute for each neuron the times when fish is in its place field
in_field_masks = falses(n_neurons, n_sweeps)

@showprogress for neuron_idx in 1:n_neurons
    
    components_peaks, img_label_valid, valid_components = map_components_peak(mean_map_all[:,:,neuron_idx]; threshold = 9/10, components_size_threshold = 20)
    
    map_mask = img_label_valid .!= 0
    
    in_field_masks[neuron_idx, :] .= Bool[map_mask[x_digital[f], y_digital[f]] for f in 1:n_sweeps]    
        
end

In [ ]:
plot(A_dFF[:, place_candidates_unique[10]])
plot(1 .* in_field_masks[place_candidates_unique[10],:])

In [ ]:
z = []
sd = []
m = []

@showprogress for neuron_idx in rand(1:n_neurons, 1000)#
    
    gap_in_s = 5

    dat = findall(1 .== in_field_masks[neuron_idx, :])

    traversals = []
    start = NaN
    for i in 1:length(dat)-1
        start = isnan(start) ? dat[i] : start

        if dat[i+1] - dat[i] > gap_in_s*2
            if dat[i] - start > 2
                append!(traversals, [[start, dat[i]]])
            end
            start = NaN
        end
    end

    act = Float32[nanmaximum(A_dFF[t[1]:t[2], neuron_idx]) for t in traversals[2:end]]

    
    means = Float32[]
    for i in 1:100
        act_control = Float32[]
        for t in traversals[2:end]
            start = rand(1:n_sweeps-15)
            duration = rand(5:15)
            append!(act_control, Float32[nanmaximum(A_dFF[start:(start+duration), neuron_idx])])
        end
        append!(means, Float32[mean(act_control[.!isnan.(act_control)])])
    end
    append!(z, [(nanmean(act) - mean(means))/std(means)])
    append!(sd, [nanstd(act)])
    append!(m, [nanmean(act)])
    
end

z2 = []
sd2 = []
m2 = []

@showprogress for neuron_idx in place_candidates_unique
    
    gap_in_s = 5

    dat = findall(1 .== in_field_masks[neuron_idx, :])

    traversals = []
    start = NaN
    for i in 1:length(dat)-1
        start = isnan(start) ? dat[i] : start

        if dat[i+1] - dat[i] > gap_in_s*2
            if dat[i] - start > 2
                append!(traversals, [[start, dat[i]]])
            end
            start = NaN
        end
    end

    act = Float32[nanmaximum(A_dFF[t[1]:t[2], neuron_idx]) for t in traversals[2:end]]

    
    means = Float32[]
    for i in 1:100
        act_control = Float32[]
        for t in traversals[2:end]
            start = rand(1:n_sweeps-15)
            duration = rand(5:15)
            append!(act_control, Float32[nanmaximum(A_dFF[start:(start+duration), neuron_idx])])
        end
        append!(means, Float32[mean(act_control[.!isnan.(act_control)])])
    end
    append!(z2, [(nanmean(act) - mean(means))/std(means)])
    append!(sd2, [nanstd(act)])
    append!(m2, [nanmean(act)])
    
end

In [ ]:
figure(figsize=(3,2))
hist(z, bins=1:0.5:25);
hist(z2, bins=1:0.5:25, alpha=0.5);
xlim(0,25); xlabel("z-score"); ylabel("cells")

In [ ]:
figure(figsize=(3,2))
hist(sd, bins=0:0.05:2);
hist(sd2, bins=0:0.05:2, alpha=0.5);

xlabel("sd"); ylabel("cells")

In [ ]:
figure(figsize=(3,2))
hist(m, bins=0:0.05:4);
hist(m2, bins=0:0.05:4, alpha=0.5);

xlabel("mean"); ylabel("cells")

In [ ]:
# firing variability for one fish
figure(figsize=(2,1.5))
scatter(m, sd, s=0.5); 
scatter(m2, sd2, s=0.5); 
xlim(0, 6); ylim(0, 4);
yticks([0, 1, 2, 3, 4])#, [1, "", "", "", 4])
xlabel("Mean firing (dF/F)"); ylabel("SD (dF/F)")
tight_layout(pad=0.5)
#savefig("firing_variability.pdf", format="pdf",  transparent=true, dpi=300)

## all datasets

In [ ]:
sd_all = []
mean_all = []
z_all_ = []


for i in 1:n_datasets
    
    
    experiment_filename = datasets_corner_cue[i][1]
    server = datasets_corner_cue[i][2]
    experimenter = datasets_corner_cue[i][3]

    ds_Lorenz = Dataset(experiment_filename, "lorenz", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
    ds_Chuyu = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(chuyu_server[i])" ? "/data" : "/nfs/data$(chuyu_server[i])")

    ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")


    C, heading, img_bg, y_fish, x_offset, x_fish, y_offset = h5open(ds, "behavior.h5"; raw = true) do file
        read(file, "C"),
        read(file, "heading"),
        read(file, "img_bg"),
        read(file, "fish_yolk_y"),
        read(file, "offset_x"),
        read(file, "fish_yolk_x"),
        read(file, "offset_y")
    end;



    # orientation-corrected fish location (time binned)
    position_file = h5open(joinpath(data_path(ds_Chuyu), "for_place_calculation_chamber_geometry_$(experiment_filename)_n60.h5"))
        chamber_roi = read(position_file,"chamber_roi")
        x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
        y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
        speed_mm_s = read(position_file, "speed_mm_s")
        loc_digital = read(position_file, "loc_digital")
        x_digital = read(position_file, "x_digital")
        y_digital = read(position_file, "y_digital")
        x_bins = read(position_file, "x_bins")
        y_bins = read(position_file, "y_bins")
    close(position_file)

    moving_valid = speed_mm_s .> 0.1;

    min_x = floor(Int64, minimum(x_fish_sweep_mean));
    max_x = floor(Int64, maximum(x_fish_sweep_mean));

    min_y = floor(Int64, minimum(y_fish_sweep_mean));
    max_y = floor(Int64, maximum(y_fish_sweep_mean));

    interval = maximum([(max_y-min_y+2)/n_pos,(max_x-min_x+2)/n_pos])


    file = h5open(ds_Chuyu, "neuron_spatial_info_15_$(lengths[i])_chamber_geometry_$(experiment_filename)_sigma1_n60_A_dF.h5", "r")
        valid_neurons = HDF5.readmmap(file["valid_neurons"])
        specificity_shuffle_z = HDF5.readmmap(file["specificity_shuffle_z"])
        specificity_population_z = HDF5.readmmap(file["specificity_population_z"])
        specificity = HDF5.readmmap(file["specificity"])
        bool_index = HDF5.readmmap(file["bool_index"])
         place_cell_index = HDF5.readmmap(file["place_cell_index"])
    close(file)

    bool_index = BitArray(bool_index)



    file = h5open(joinpath(data_path(ds_Chuyu), "NMF_merge.h5"), "r")
        #A_dF = HDF5.readmmap(file["A_dF"])
        A_dFF = HDF5.readmmap(file["A_dF"])
        neuron_label = HDF5.readmmap(file["neuron_label"])

        z_all = HDF5.readmmap(file["Z_all"])
        centroid_x_all = HDF5.readmmap(file["X_all"])
        centroid_y_all = HDF5.readmmap(file["Y_all"])
    close(file)

    n_neurons = size(A_dFF, 2)
    n_sweeps = size(A_dFF, 1)
    
    
    
        
    file = h5open(joinpath(data_path(ds_Chuyu), "region_roi_bool.h5"))
        region_names = read(file, "region_names")
        region_roi_bool = read(file, "region_roi_bool")
    close(file)


    tel_index = findall(region_names .== "Telencephalon -")[1];
    mask_tel = falses(n_neurons)

    for which_neuron in 1:n_neurons
        mask_tel[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, tel_index])
    end

    mask_tel = findall(mask_tel)

    place_cell_index = intersect(place_cell_index, mask_tel)
    



    function bin_to_px(x, y, offset=true)
        if offset
            return (x .- 0.5) .* interval .+ (min_x-1), (y .- 0.5) .* interval .+ (min_y-1)
        else
            return (x .- 0.5) .* interval, (y .- 0.5) .* interval
        end
    end

    function px_to_bin(x, y)
        return 0.5 .+ ((x .- (min_x-1)) ./ interval), 0.5 .+ ((y .- (min_y-1)) ./ interval)
    end

    x_in_bins, y_in_bins = px_to_bin(x_fish_sweep_mean, y_fish_sweep_mean);

    try
        mkdir(data_path(ds_Lorenz))    
    catch
        println("save path exists")
    end

    #place_candidates_unique = Decoder.get_top_neurons(use_amount, specificity_population_z, specificity_shuffle_z);
    
    
    mean_map_all = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
    count_map_all = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
    @showprogress for neuron_idx in place_cell_index#1:size(A_dFF, 2)

        neural_activity, which_loc = MAP.valid_activity_loc(A_dFF[:, neuron_idx], bool_index, loc_digital)
        mean_map_all[:, :, neuron_idx], count_map_all[:, :, neuron_idx], summed = MAP.calculate_map_direct(neural_activity, which_loc, n_pos; at_least_visit = 0, use_gaussian_filter=true, sigma=1, filter_mask = nothing)

    end
    
    
    
    function whether_in(vector, collection)
    return [x in collection for x in vector]
end

in_field_masks = falses(n_neurons, n_sweeps)

@showprogress for neuron_idx in place_cell_index#1:n_neurons# place_candidates_unique
    
    components_peaks, img_label_valid, valid_components = map_components_peak(mean_map_all[:,:,neuron_idx]; threshold = 9/10, components_size_threshold = 20)
    
    map_mask = img_label_valid .!= 0
    
    in_field_masks[neuron_idx, :] .= Bool[map_mask[x_digital[f], y_digital[f]] for f in 1:n_sweeps]    
        
end
    

    z2 = []
    sd2 = []
    m2 = []

    @showprogress for neuron_idx in place_cell_index#place_candidates_unique

        gap_in_s = 5

        dat = findall(1 .== in_field_masks[neuron_idx, :])

        traversals = []
        start = NaN
        for i in 1:length(dat)-1
            start = isnan(start) ? dat[i] : start

            if dat[i+1] - dat[i] > gap_in_s*2
                if dat[i] - start > 2
                    append!(traversals, [[start, dat[i]]])
                end
                start = NaN
            end
        end

        act = Float32[nanmaximum(A_dFF[t[1]:t[2], neuron_idx]) for t in traversals[2:end]]
        
        
        means = Float32[]
        for i in 1:100
            act_control = Float32[]
            for t in traversals[2:end]
                start = rand(1:n_sweeps-15)
                duration = rand(5:15)
                append!(act_control, Float32[nanmaximum(A_dFF[start:(start+duration), neuron_idx])])
            end
            append!(means, Float32[mean(act_control[.!isnan.(act_control)])])
        end
        append!(z2, [(nanmean(act) - mean(means))/std(means)])
        append!(sd2, [nanstd(act)])
        append!(m2, [nanmean(act)])

    end
    append!(z_all_, [z2])
    append!(sd_all, [sd2])
    append!(mean_all, [m2])
end


In [ ]:
figure(figsize=(1.5,1.5))
scatter(vcat(mean_all...), vcat(sd_all...), s=0.1, alpha=0.1, c="black");
xlim(0, 600); ylim(0, 400);
yticks([0, 600])
xticks([0, 600])
xlabel("Mean activity (dF)", labelpad=-3); ylabel("Standard dev. (dF)", labelpad=-5)
tight_layout(pad=0.5)
plot([0,600],[0,600], color="grey", linestyle="dashed", linewidth=0.7)
savefig("EDF1e_firing_variability.pdf", format="pdf",  transparent=true, dpi=300)